In [ ]:
################################################
#
# H U M A N 
#
################################################

organism = 'Human'

G = nx.read_edgelist('input/ppi_elist.txt',data=False)
# d_ent_sym, d_sym_ent = genent2sym()

d_gene_do = pickle.load( open( "input/d_gene_do.pkl", "rb" ) )
d_do_genes = pickle.load( open( "input/d_do_genes.pkl", "rb" ) )
d_do_names = pickle.load( open( "input/DO_names.pkl", "rb" ) )
d_names_do = {y:x for x,y in d_do_names.items()}

In [ ]:
entrez_syn = pd.read_csv('input/entrezid_syn.csv', index_col=0)
d_entrez_syn = dict(zip(entrez_syn.index,entrez_syn['db_synonym']))

d_gene2syn = {}
for k,v in d_entrez_syn.items():
    try:
        if k in G.nodes():
            d_gene2syn[k] = v
    except: 
           d_gene2syn['None'] = 'no entry'
            
d_genes_withsyn = {}
for g in G.nodes():
    for k,v in d_gene2syn.items():
        if g == k:
            d_genes_withsyn[g] = v
            
d_genes_nosyn = {}
for gene in G.nodes():
    if gene not in d_genes_withsyn.keys():
        d_genes_nosyn[gene] = 'no entry'
        
d_genes_all = {**d_genes_withsyn, **d_genes_nosyn}
d_genes_syn_sorted = {key:d_genes_all[key] for key in G.nodes()}

l_genes_syn=[]
for k,v in d_genes_syn_sorted.items():
    try:
        l_genes_syn.append(set(v.split('|')))
    except:
        l_genes_syn.append(v)
        
d_genes_syn_complete = {}
for i,v in enumerate(d_genes_syn_sorted.keys()):
    for idx,l in enumerate(l_genes_syn):
        if i==idx:
            d_genes_syn_complete[v] = l
            
l_features = list(d_genes_syn_complete.values())

### IBD Variants from Julia 

In [ ]:
IBD_genes = np.loadtxt("input/iei_hubness.txt", usecols=(0, 1), skiprows=1, dtype=str)

d_IBD_genes = {}
for i in IBD_genes:
    d_IBD_genes[i[0]] = i[1]
    
    
d_degs = dict(G.degree())
d_degs_sortbydegree = {k: v for k, v in sorted(d_degs.items(), key=lambda item: item[1], reverse=True)}

# dict with gene to syn
d_geneid_IBDsyn = {}
for g in d_IBD_genes.keys():
    for k,v in d_genes_syn_complete.items():
        try:
            for gene in v:
                if g == gene:
                    d_geneid_IBDsyn[k] = g
        except:
            pass
    
# dict with gene to syn
d_IBDsyn_geneid = {}
for g in d_IBD_genes.keys():
    for k,v in d_genes_syn_complete.items():
        try:
            for gene in v:
                if g == gene:
                    d_IBDsyn_geneid[g] = k
        except:
            pass

# DISEASE Association with IBD genes 

d_IBD_doids = {}
for g,doids in d_gene_do.items():
    for idx,syn in d_geneid_IBDsyn.items():
        if g == idx: 
            d_IBD_doids[syn] = doids